In [2]:
import json
import geopandas as gpd

#INPUT VARIABLES
config_path = '00_input/korindo.json'
is_export_image_to_drive = False

# to-do, filter the AOI based on size, if the AOI is too big, we will use the GEE approach, if small, we will use the STAC approach
with open(config_path, 'r') as f:
    config = json.load(f)

aoi_gpd = gpd.GeoDataFrame.from_file(config["AOI_path"])
aoi_gpd = aoi_gpd.to_crs(epsg=int(config["output_crs"].split(":")[-1])) # satellite crs is epsg code of projected UTM crs
aoi_ha = aoi_gpd.geometry.area.sum()/10000

if aoi_ha > 5000: # 5,000 is the threshold for the AOI size
    print(f"AOI area ({aoi_ha} ha) is too big, using GEE approach when available and mix with STAC MPC")
    use_gee = True
else:
    print(f"AOI area ({aoi_ha} ha) is small, using STAC (local process xarray) approach")
    use_gee = False

AOI area (144217.6737820076 ha) is too big, using GEE approach when available and mix with STAC MPC


In [3]:
!pip install gcsfs

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from forestry_carbon_arr.utils.zarr_utils import load_dataset_zarr
import os

zarr_path = os.getenv('GCS_ZARR_DIR') + '/ds_resampled.zarr'
# zarr_path = 'data/ds_resampled.zarr'
# storage = 'local'
storage = 'gcs'

ds_resampled = load_dataset_zarr(zarr_path, storage=storage)
ds_resampled

📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/timeseries_zarr/ds_resampled.zarr
✅ Dataset loaded: {'time': 81, 'x': 4489, 'y': 3213}


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:730: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


<xarray.Dataset> Size: 9GB
Dimensions:   (time: 81, x: 4489, y: 3213)
Coordinates:
    image_id  (time) <U17 6kB dask.array<chunksize=(40,), meta=np.ndarray>
  * time      (time) datetime64[ns] 648B 2018-02-15 2018-05-15 ... 2025-08-15
  * x         (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y         (y) float64 26kB 9.949e+06 9.949e+06 ... 9.982e+06 9.982e+06
Data variables:
    EVI       (time, x, y) float32 5GB dask.array<chunksize=(40, 1024, 1024), meta=np.ndarray>
    NDVI      (time, x, y) float32 5GB dask.array<chunksize=(40, 1024, 1024), meta=np.ndarray>
Attributes:
    crs:      EPSG:32749

In [11]:
import ee, eemont
import pandas as pd
import numpy as np
from forestry_carbon_arr.core import ForestryCarbonARR

forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')

# Use the renamed method
results = forestry.get_ds_sampled_mpc(
    zarr_path=zarr_path,
    create_visualization=True,
    # save_map=True,  # Opt-in to save HTML
    save_map=False,
    map_filename='my_map.html'  # Optional custom filename # ignore when save_map is False
)

# Or save manually later
# results['visualization'].save('map.html')

📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/timeseries_zarr/ds_resampled.zarr
✅ Dataset loaded: {'time': 81, 'x': 4489, 'y': 3213}


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:730: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


Creating visualization map...
🗺️ CREATING UNIFIED STAC VISUALIZATION
Adding STAC scene versions: S2A_MSIL2A_20171026T023801_R089_T49MEV
Found 2 versions to overlay
   1. S2A_MSIL2A_20171026T023801_R089_T49MEV_20201015T021831 - green boundary
   2. S2A_MSIL2A_20171026T023801_R089_T49MEV_20201015T021828 - orange boundary
✅ Added 2 STAC scene version layers
Returning folium map object: <class 'folium.folium.Map'>
✅ Visualization created
Visualization type: <class 'folium.folium.Map'>
🔗 MICROSOFT PLANETARY COMPUTER URLs
📍 Your area center: -0.3125, 111.9082
📍 Bbox: (111.706493374869, -0.457839253478653, 112.109836017418, -0.167195667545911)

🛰️ Sentinel-2 L2A (Natural Color):
  https://planetarycomputer.microsoft.com/explore?c=111.9081646961435%2C-0.312517460512282&z=12&v=2&d=sentinel-2-l2a&s=false%3A%3A100%3A%3Atrue&ae=0&sr=desc&m=Most+recent+%28low+cloud%29&r=Natural+color

🛰️ Sentinel-2 L2A (False Color):
  https://planetarycomputer.microsoft.com/explore?c=111.9081646961435%2C-0.3125174

In [6]:
# Display the geemap directly
results['visualization']

In [7]:
# results['ds_resampled']

In [8]:
# results['mpc_date_range']

In [9]:
results['mpc_urls']

{'🛰️ Sentinel-2 L2A (Natural Color)': 'https://planetarycomputer.microsoft.com/explore?c=111.9081646961435%2C-0.312517460512282&z=12&v=2&d=sentinel-2-l2a&s=false%3A%3A100%3A%3Atrue&ae=0&sr=desc&m=Most+recent+%28low+cloud%29&r=Natural+color',
 '🛰️ Sentinel-2 L2A (False Color)': 'https://planetarycomputer.microsoft.com/explore?c=111.9081646961435%2C-0.312517460512282&z=12&v=2&d=sentinel-2-l2a&s=false%3A%3A100%3A%3Atrue&ae=0&sr=desc&m=Most+recent+%28low+cloud%29&r=Short+wave+infrared',
 '🛰️ Landsat 8-9 (Natural Color)': 'https://planetarycomputer.microsoft.com/explore?c=111.9081646961435%2C-0.312517460512282&z=12&v=2&d=landsat-c2-l2&s=false%3A%3A100%3A%3Atrue&ae=0&sr=desc&m=Most+recent+%28low+cloud%29&r=Natural+color'}

In [10]:
# ds_resampled

In [11]:
# os.getenv('GCS_ZARR_DIR')

In [12]:
results['mpc_date_range']

['2014-08-15', '2018-01-31']

In [13]:
### Download Sentinel-2 satellite data from Microsoft Planetary Computer (MPC)
### This uses the bbox and date_range_mpc from the previous get_ds_sampled_mpc results

# Get bbox and date_range_mpc from results
bbox = results['bbox']
date_range_mpc = results['mpc_date_range']
date_range_str = results['mpc_date_range'][0]+'_'+results['mpc_date_range'][1]
satellite_type = forestry.config['collection_mpc']
project_name = forestry.config['project_name']

zarr_path = os.path.join(os.getenv('GCS_ZARR_DIR'), f'{satellite_type}_{project_name}_{date_range_str}.zarr')

print("📡 Using STAC (Microsoft Planetary Computer) approach...")
print(f"📅 Date range: {date_range_mpc}")

# Customize chunk sizes for zarr storage
# Default is {'time': 10, 'x': 512, 'y': 512}
# Adjust based on your data size and memory constraints
chunk_sizes = {
    'time': 10,   # Number of time steps per chunk
    'x': 512,     # Number of pixels in x direction per chunk
    'y': 512      # Number of pixels in y direction per chunk
}

# Download and process satellite data
ds_sentinel_mpc = forestry.get_satellite_mpc(
    bbox=bbox,  # Can be None - will use from previous results
    date_range_mpc=date_range_mpc,  # Can be None - will use from previous results
    utm_crs=forestry.config['output_crs'],  # can be auto from bbox too
    pixel_scale=forestry.config['resolution_satellite'],  # Auto-detect (10m for Sentinel-2)
    zarr_path=zarr_path,  # Custom path with date range and satellite type
    chunk_sizes=chunk_sizes,  # Custom chunk sizes for zarr storage
    save_to_zarr=True,  # Save to zarr
    storage='auto',  # Auto-detect (GCS if GCS_ZARR_DIR env var is set)
    show_progress=True
)

print(f"\n✅ Dataset downloaded: {dict(ds_sentinel_mpc.sizes)}")
print(f"📊 Data variables: {list(ds_sentinel_mpc.data_vars)}")

### for local zarr saving
# ds_sentinel_mpc = forestry.get_satellite_mpc(
#     bbox=bbox,
#     date_range_mpc=date_range_mpc,
#     zarr_path='data/my_custom_folder/sentinel2.zarr',  # Custom local path
#     storage='local'
# )

# # Default: GEE-compatible (Zarr v2, lz4, consolidated metadata)
# save_dataset_efficient_zarr(ds, 'gs://bucket/data.zarr')

# # GEE-compatible with custom compression
# save_dataset_efficient_zarr(
#     ds, 'gs://bucket/data.zarr',
#     compression='zstd',  # GEE supports zstd
#     gee_compatible=True  # Still enforces Zarr v2
# )

# # Non-GEE format (can use Zarr v3, any compression, etc.)
# save_dataset_efficient_zarr(
#     ds, 'gs://bucket/data.zarr',
#     compression='blosclz',  # Not GEE-compatible but OK for other uses
#     zarr_version=3,
#     gee_compatible=False
# )

📡 Using STAC (Microsoft Planetary Computer) approach...
📅 Date range: ['2014-08-15', '2018-01-31']
🛰️  DOWNLOADING SATELLITE DATA FROM MPC
📅 Date range: 2014-08-15/2018-01-31
📥 Zarr not found. Proceeding with download...
📡 Processing satellite data from STAC...
🚀 Starting simple STAC satellite data processing pipeline...
📊 AOI Area: 144217.7 hectares
🔍 Step 1/5: Searching for satellite data...
✅ Found 71 images in 1.2s
📦 Step 2/5: Creating data stack...
✅ Data stack created in 0.6s
☁️  Step 3/5: Applying cloud masking...
✅ Cloud masking applied in 0.0s
🏷️  Step 4/5: Renaming bands...
✅ Bands renamed in 0.0s
💾 Step 5/5: Computing and downloading data...
   📡 Downloading satellite imagery...
🎉 STAC Processing Complete, please proceed to download to local first, and also to the gcp_bucket!
⏱️  Total time: 0.0 minutes (1.9s)
✅ STAC data processed: {'time': 71, 'y': 3213, 'x': 4489}
💾 Saving to zarr: gs://remote_sensing_saas/01-korindo/timeseries_zarr/sentinel-2-l2a_korindo_2014-08-15_2018-

/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:231: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"   Dimensions: {dict(ds.dims)}")


💾 Writing to zarr (with automatic parallelism)...


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:263: FutureWarning: zarr_version is deprecated, use zarr_format
  ds_chunked.to_zarr(


[########################################] | 100% Completed | 24m 24s
✅ Dataset saved successfully!
   Store size: 7.19 GB
   Time: 1481.6 seconds (24.7 minutes)
   Write speed: 5.0 MB/s
   Path: gs://remote_sensing_saas/01-korindo/timeseries_zarr/sentinel-2-l2a_korindo_2014-08-15_2018-01-31.zarr
✅ Dataset saved to zarr
✅ MPC SATELLITE DATA DOWNLOAD COMPLETE

✅ Dataset downloaded: {'time': 71, 'y': 3213, 'x': 4489}
📊 Data variables: ['coastal', 'blue', 'green', 'red', 'redE1', 'redE2', 'redE3', 'nir', 'nir2', 'swir1', 'swir2', 'redE4', 'scl', 'is_valid']


In [12]:
import json

#INPUT VARIABLES
config_path = '00_input/korindo.json'

with open(config_path, 'r') as f:
    config = json.load(f)

config

{'I_satellite': 'Sentinel',
 'collection_mpc': 'sentinel-2-l2a',
 'pca_scaling': 1,
 'tileScale': 2,
 'AOI_path': './00_input/aoi_korindo.shp',
 'OID_field_name': 'id',
 'algo_ml_selected': 'gbm',
 'date_start_end': ['2025-8-1', '2025-8-31'],
 'project_name': 'korindo',
 'super_pixel_size': 3,
 'region': 'asia',
 'pixel_number': 3,
 'year_start_loss': 14,
 'tree_cover_forest': 30,
 'band_name_image': 'Class',
 'cloud_cover_threshold': 80,
 'valid_pixel_threshold': 20,
 'crs_input': 'EPSG:4326',
 'output_crs': 'EPSG:32749',
 'IsThermal': False,
 'fcd_selected': 21,
 'high_forest': 65,
 'yrf_forest': 45,
 'shrub_grass': 45,
 'open_land': 30,
 'ndwi_hi_sentinel': 0.05,
 'ndwi_hi_landsat': 0.1,
 'ndwi_hi_planet': -0.2,
 'resolution_satellite': 10,
 'assets_satellite': ['B01',
  'B02',
  'B03',
  'B04',
  'B05',
  'B06',
  'B07',
  'B08',
  'B09',
  'B11',
  'B12',
  'B8A',
  'SCL'],
 'band_mapping': {'B01': 'coastal',
  'B02': 'blue',
  'B03': 'green',
  'B04': 'red',
  'B05': 'redE1',
  '

In [15]:
config['mpc_date_range'] = date_range_mpc
config['mpc_zarr_path'] = zarr_path

with open(config_path, 'w') as f:
    json.dump(config, f)

In [16]:
ds_sentinel_mpc

<xarray.Dataset> Size: 108GB
Dimensions:    (time: 71, y: 3213, x: 4489)
Coordinates:
  * time       (time) datetime64[ns] 568B 2015-11-26T02:40:32.031000 ... 2017...
    id         (time) <U54 15kB 'S2A_MSIL2A_20151126T024032_R089_T49MFV_20210...
  * x          (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y          (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
    platform   (time) <U11 3kB 'Sentinel-2A' 'Sentinel-2A' ... 'Sentinel-2A'
    proj:code  <U10 40B 'EPSG:32749'
    epsg       int64 8B 32749
Data variables: (12/14)
    coastal    (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    blue       (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    green      (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    red        (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    redE1      (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    redE2      (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    ...         ...
    nir2       (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    swir1      (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    swir2      (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    redE4      (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    scl        (time, y, x) float64 8GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
    is_valid   (time, y, x) bool 1GB dask.array<chunksize=(1, 199, 327), meta=np.ndarray>
Attributes:
    spec:        RasterSpec(epsg=32749, bounds=(499970, 9890150, 709840, 1000...
    crs:         epsg:32749
    transform:   | 10.00, 0.00, 499970.00|\n| 0.00,-10.00, 10000000.00|\n| 0....
    resolution:  10

In [15]:
# Convert zarr to GEE-compatible format
ds_gee = forestry.convert_zarr_for_gee(
    zarr_path=config['mpc_zarr_path'],
    output_path=os.path.join(config['mpc_zarr_path'].replace('.zarr', '_gee.zarr')), # Optional: new path
    storage='auto',
    overwrite=True
)

print(f"Success: {ds_gee['success']}")
print(f"GEE URI: {ds_gee['gee_uri']}")
print(f"Requirements met: {ds_gee['requirements_met']}")
print(f"Warnings: {ds_gee['warnings']}")

📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/timeseries_zarr/sentinel-2-l2a_korindo_2014-08-15_2018-01-31.zarr


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:734: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:261: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if 'crs' in ds.attrs:
/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:323: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  compression='lz4',  # GEE-compatible
/usr/src/app/forestry_car

✅ Dataset loaded: {'time': 71, 'y': 3213, 'x': 4489}
🔄 Converting existing zarr to GEE-compatible format...
   Input: gs://remote_sensing_saas/01-korindo/timeseries_zarr/sentinel-2-l2a_korindo_2014-08-15_2018-01-31.zarr
   Output: gs://remote_sensing_saas/01-korindo/timeseries_zarr/sentinel-2-l2a_korindo_2014-08-15_2018-01-31_gee.zarr
   Variables: ['blue', 'coastal', 'green', 'is_valid', 'nir', 'nir2', 'red', 'redE1', 'redE2', 'redE3', 'redE4', 'scl', 'swir1', 'swir2']
   Dimensions: ['time', 'y', 'x']
🗑️  Removing existing GCS zarr store: gs://remote_sensing_saas/01-korindo/timeseries_zarr/sentinel-2-l2a_korindo_2014-08-15_2018-01-31_gee.zarr
📦 Saving to zarr: gs://remote_sensing_saas/01-korindo/timeseries_zarr/sentinel-2-l2a_korindo_2014-08-15_2018-01-31_gee.zarr
   Dimensions: {'time': 71, 'y': 3213, 'x': 4489}
   Chunks: {'time': 10, 'y': 512, 'x': 512}
   Compression: lz4 (level 1)
   Storage: gcs
💾 Writing to zarr (with automatic parallelism)...
[################################

In [16]:
## only if we convert the zarr to GEE-compatible format
config['mpc_zarr_path'] = os.path.join(config['mpc_zarr_path'].replace('.zarr', '_gee.zarr'))

with open(config_path, 'w') as f:
    json.dump(config, f)